<a href="https://colab.research.google.com/github/jayshah1819/tensorrt-int8-practice/blob/main/INT8_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!dpkg -l | grep nvinfer


iU  libnvinfer-headers-dev                 10.13.3.9-1+cuda13.0                    amd64        TensorRT development headers
iHR libnvinfer10                           10.13.3.9-1+cuda13.0                    amd64        (no description available)


In [4]:
!pip install tensorrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.3 MB/s eta 0:00:00
  Created wheel for tensorrt: filename=tensorrt-10.13.3.9-py2.py3-none-any.whl size=46401 sha256=7c01d2355b497d8c6b32800c3a2aae49d2666ffc3eb5f4c55ace3fb63da97d85
  Stored in directory: /root/.cache/pip/wheels/1a/aa/b0/561559321c1659e43c5a6661986c3072f9c383efeeaaffb1a5
  Created wheel for tensorrt_cu13: filename=tensorrt_cu13-10.13.3.9-py2.py3-none-any.whl size=17436 sha256=62a3918754cbe53d4b725720a01bdebebcf394c5568996078af95be766e180e6
  Stored in directory: /root/.cache/pip/wheels/df/4a/b2/ebfc5437a397c53faff0dca4a36c096d4b7d2b7436a2707e60
  Created wheel for tensorrt_cu13_libs: filename=tensorrt_cu13_libs-10.13.3.9-

Step 1) Defining the network

Step 2) Setting up the INT8 calibrator

Step 3) Configuring the builder

Step 4) Building the TensorRT engine

Step 5) Running inference

Step 6) Verifying the output **bold text**

In [6]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 11.4 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1.2-cp312-cp312-linux_x86_64.whl size=659050 sha256=4a951a06e4ab10659acf6c4b7657b889abcd4c6a4659ecfdf1d2fd5f3bbd6869
  Stored in directory: /root/.cache/pip/wheels/d5/36/f3/ac5f09d768cad3fa15d5a3449bdfe65c3de58e69d036c73228
Successfully built pycuda


In [7]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np


 Defining the network

1.   Prepare Representative Dataset
2.   Create INT8 Calibrator





In [8]:
calibration_images=np.random.rand(32,3,224,224).astype(np.float32)

In [9]:
class MyCalibrator(trt.IInt8EntropyCalibrator2):
    def __init__(self, data, batch_size=8):

        super().__init__()
        self.data = data
        self.batch_size = batch_size
        self.index = 0
        self.device_input = None

    def get_batch_size(self):
        return self.batch_size

    def get_batch(self, names):
        if self.index + self.batch_size > len(self.data):
            return None
        batch = self.data[self.index:self.index+self.batch_size].ravel()
        if self.device_input is None:
            self.device_input = cuda.mem_alloc(batch.nbytes)
        cuda.memcpy_htod(self.device_input, batch)
        self.index += self.batch_size
        return [int(self.device_input)]

    def read_calibration_cache(self):
        return None

    def write_calibration_cache(self, cache):
        pass
